<h2 align="center" style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Forecasting Stock Prices using Sentiment Analysis and Time Series Models: An Advanced Data Analytics Approach</h2>

### **Table of Contents**

- [Introduction](#Introduction)
   - Assessment Overview
   - Objectives
   - Data Source and Storage
- [Install and Import Required Libraries](#Install-and-Import-Required-Libraries)
- [Load Dataset](#Load-Dataset)

<h3 style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Introduction</h3>

### Assessment Overview

### Objectives

### Data Source and Storage

<h3 style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Install and Import Required Libraries</h3>

In [1]:
!pip install -q pyspark pandas

In [2]:
import os
from datetime import datetime, timedelta

import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, lit, to_date, avg, stddev, desc, first
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
from pyspark.sql.window import Window

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

<h3 style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Load Dataset</h3>

#### **Initialize Spark Session and Define Data Paths**

In [3]:
spark = SparkSession.builder \
    .appName("Stock Tweet Analysis") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "10g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/21 10:21:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.setLogLevel("ERROR")

In [5]:
tweet_data_path = "/kaggle/input/stock-tweet-and-price/stock-tweet-and-price/stocktweet/stocktweet.csv"
stock_price_folder = "/kaggle/input/stock-tweet-and-price/stock-tweet-and-price/stockprice"
db_path = "stock_analysis.db"

#### **Define Data Schemas**

In [6]:
# Define the companies to analyze
companies = ['AAPL', 'AMZN', 'MSFT', 'TSLA', 'GOOGL', 'FB']

In [7]:
# Define schema for tweet data
tweet_schema = StructType([
    StructField("id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("ticker", StringType(), True),
    StructField("tweet", StringType(), True)
])

In [8]:
# Define schema for stock price data
stock_schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Open", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Low", DoubleType(), True),
    StructField("Close", DoubleType(), True),
    StructField("Adj Close", DoubleType(), True),
    StructField("Volume", LongType(), True)
])

#### **Define Helper Functions for Loading Data**

In [9]:
def load_tweet_data():
    df = spark.read.csv(tweet_data_path, header=True, schema=tweet_schema)
    # Convert date string to standard format
    df = df.withColumn("date", to_date(col("date"), "MM/dd/yyyy"))
    # Filter tweets for selected companies
    df = df.filter(col("ticker").isin(companies))
    return df

In [10]:
def load_stock_data(ticker):
    file_path = os.path.join(stock_price_folder, f"{ticker}.csv")
    df = spark.read.csv(file_path, header=True, schema=stock_schema)
    # Convert date string to standard format
    df = df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))
    # Add ticker column
    df = df.withColumn("ticker", lit(ticker))
    return df

#### **Load Datasets using Helper Functions**

In [11]:
print("Loading tweet data...")
tweets_df = load_tweet_data()
print("Tweet data loaded")

Loading tweet data...
Tweet data loaded


In [12]:
print("Loading stock price data...")
stock_dfs = {}
for company in companies:
    stock_dfs[company] = load_stock_data(company)
print("Stock price data loaded")

Loading stock price data...
Stock price data loaded


<h3 style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Data Exploration</h3>

#### **Viewing First 5 Rows of Each Stock DataFrame**

In [13]:
def show_top_rows(df, name):
    """Display the top 5 rows of a DataFrame"""
    print(f"\n{name} Top 5 Rows:")
    df.show(5, truncate=False)

In [14]:
for ticker, df in stock_dfs.items():
    show_top_rows(df, ticker)


AAPL Top 5 Rows:


+----------+-----------------+-----------------+-----------------+-----------------+-----------------+---------+------+
|Date      |Open             |High             |Low              |Close            |Adj Close        |Volume   |ticker|
+----------+-----------------+-----------------+-----------------+-----------------+-----------------+---------+------+
|2019-12-31|72.48249816894531|73.41999816894531|72.37999725341797|73.4124984741211 |71.52082061767578|100805600|AAPL  |
|2020-01-02|74.05999755859375|75.1500015258789 |73.79750061035156|75.0875015258789 |73.15264892578125|135480400|AAPL  |
|2020-01-03|74.2874984741211 |75.1449966430664 |74.125           |74.35749816894531|72.44145965576172|146322800|AAPL  |
|2020-01-06|73.44750213623047|74.98999786376953|73.1875          |74.94999694824219|73.0186767578125 |118387200|AAPL  |
|2020-01-07|74.95999908447266|75.2249984741211 |74.37000274658203|74.59750366210938|72.67527770996094|108872000|AAPL  |
+----------+-----------------+----------

#### **Statistical Summary of Each Stock DataFrame**

In [15]:
def show_summary(df, name):
    """Display statistical summary of a DataFrame"""
    print(f"\n{name} Statistical Summary:")
    df.describe().show()

In [16]:
# spark.conf.set("spark.sql.debug.maxToStringFields", 10)

In [17]:
for ticker, df in stock_dfs.items():
    show_summary(df, ticker)


AAPL Statistical Summary:


+-------+------------------+-----------------+------------------+------------------+------------------+--------------------+------+
|summary|              Open|             High|               Low|             Close|         Adj Close|              Volume|ticker|
+-------+------------------+-----------------+------------------+------------------+------------------+--------------------+------+
|  count|               254|              254|               254|               254|               254|                 254|   254|
|   mean| 95.17796276122566|96.57026571739377| 93.82802144748958| 95.26071827805887| 93.30824790413924|1.5734118582677165E8|  NULL|
| stddev|22.014833707521472|22.09909824629524| 21.57955997463146|21.810136925990065|21.574201492769824| 6.978351522681883E7|  NULL|
|    min| 57.02000045776367|           57.125| 53.15250015258789|56.092498779296875| 54.77680206298828|            46691300|  AAPL|
|    max| 138.0500030517578|138.7899932861328|134.33999633789062|136.6900024

#### **Checking for Missing Values**

In [18]:
def check_missing_values(df, name):
    """Check missing values in a Dataframe"""
    print(f"\nMissing Values in {name}:")
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [19]:
for ticker, df in stock_dfs.items():
    check_missing_values(df, ticker)


Missing Values in AAPL:
+----+----+----+---+-----+---------+------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|ticker|
+----+----+----+---+-----+---------+------+------+
|   0|   0|   0|  0|    0|        0|     0|     0|
+----+----+----+---+-----+---------+------+------+


Missing Values in AMZN:
+----+----+----+---+-----+---------+------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|ticker|
+----+----+----+---+-----+---------+------+------+
|   0|   0|   0|  0|    0|        0|     0|     0|
+----+----+----+---+-----+---------+------+------+


Missing Values in MSFT:
+----+----+----+---+-----+---------+------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|ticker|
+----+----+----+---+-----+---------+------+------+
|   0|   0|   0|  0|    0|        0|     0|     0|
+----+----+----+---+-----+---------+------+------+


Missing Values in TSLA:
+----+----+----+---+-----+---------+------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|ticker|
+----+----+----+---+-----+----

#### **Explore tweet data**

In [20]:
show_top_rows(tweets_df, "Tweet Data")
show_summary(tweets_df, "Tweet Data")
check_missing_values(tweets_df, "Tweets")


Tweet Data Top 5 Rows:
+------+----------+------+-------------------------------------------------------------------------------------------------------------------------------------------+
|id    |date      |ticker|tweet                                                                                                                                      |
+------+----------+------+-------------------------------------------------------------------------------------------------------------------------------------------+
|100001|2020-01-01|AMZN  |$AMZN Dow futures up by 100 points already 🥳                                                                                              |
|100002|2020-01-01|TSLA  |$TSLA Daddy's drinkin' eArly tonight! Here's to a PT of ohhhhh $1000 in 2020! 🍻                                                           |
|100003|2020-01-01|AAPL  |$AAPL We’ll been riding since last December from $172.12 what to do. Decisions decisions hmm 🤔. I have 20 mins to dec

+---+----+------+-----+
| id|date|ticker|tweet|
+---+----+------+-----+
|  0|4177|     0|    0|
+---+----+------+-----+



In [21]:
# Tweet counts
print("\nTweet Count by Company:")
tweets_df.groupBy("ticker").count().orderBy(desc("count")).show()

print("\nTweet Count by Date (Top 10):")
tweets_df.groupBy("date").count().orderBy(desc("count")).show(10)


Tweet Count by Company:
+------+-----+
|ticker|count|
+------+-----+
|  TSLA| 4341|
|  AAPL| 1721|
|  AMZN|  407|
|  MSFT|  271|
|    FB|  204|
| GOOGL|   17|
+------+-----+


Tweet Count by Date (Top 10):
+----------+-----+
|      date|count|
+----------+-----+
|      NULL| 4177|
|2020-03-09|  137|
|2020-01-09|  114|
|2020-01-05|  108|
|2020-02-09|  102|
|2020-04-09|  101|
|2020-08-09|   78|
|2020-09-09|   65|
|2020-10-09|   63|
|2020-03-03|   60|
+----------+-----+
only showing top 10 rows



<h3 style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Data Preprocessing</h3>

In [22]:
# Process tweet data - calculate daily tweet counts for each company
tweet_counts = tweets_df.groupBy("date", "ticker").count().withColumnRenamed("count", "tweet_count")

In [23]:
def preprocess_data(stock_dfs, tweets_df):
    """Process and join data for each company"""
    processed_dfs = {}
    for ticker, stock_df in stock_dfs.items():
        # Rename date column for consistent joining
        stock_df = stock_df.withColumnRenamed("Date", "date")
        
        # Join stock data with tweet counts
        joined_df = stock_df.join(
            tweet_counts.filter(col("ticker") == ticker),
            on=["date", "ticker"],
            how="left"
        )
        
        # Fill missing tweet counts with 0
        joined_df = joined_df.na.fill({"tweet_count": 0})
        
        # Store the processed DataFrame
        processed_dfs[ticker] = joined_df
        
        # Show the processed data
        print(f"\nProcessed {ticker} Data:")
        joined_df.show(5)
    
    return processed_dfs

In [24]:
processed_dfs = preprocess_data(stock_dfs, tweets_df)


Processed AAPL Data:
+----------+------+-----------------+-----------------+-----------------+-----------------+-----------------+---------+-----------+
|      date|ticker|             Open|             High|              Low|            Close|        Adj Close|   Volume|tweet_count|
+----------+------+-----------------+-----------------+-----------------+-----------------+-----------------+---------+-----------+
|2019-12-31|  AAPL|72.48249816894531|73.41999816894531|72.37999725341797| 73.4124984741211|71.52082061767578|100805600|          0|
|2020-01-02|  AAPL|74.05999755859375| 75.1500015258789|73.79750061035156| 75.0875015258789|73.15264892578125|135480400|          3|
|2020-01-03|  AAPL| 74.2874984741211| 75.1449966430664|           74.125|74.35749816894531|72.44145965576172|146322800|          1|
|2020-01-06|  AAPL|73.44750213623047|74.98999786376953|          73.1875|74.94999694824219| 73.0186767578125|118387200|          2|
|2020-01-07|  AAPL|74.95999908447266| 75.2249984741211

<h3 style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Store Data to SQL Database</h3>

In [25]:
# Create SQLite database connection
def create_db_connection():
    conn = sqlite3.connect(db_path)
    return conn

In [26]:
# Store data to SQLite using PySpark
def store_data_to_sql(tweets_df, stock_dfs, processed_dfs):
    # Create database connection
    conn = create_db_connection()
    
    # Store tweets data - convert to pandas first
    tweets_df.toPandas().to_sql("tweets", conn, if_exists="replace", index=False)
    
    # Store stock price data for each company
    for ticker, df in stock_dfs.items():
        df.toPandas().to_sql(f"stock_prices_{ticker}", conn, if_exists="replace", index=False)
    
    # Store processed data for each company
    for ticker, df in processed_dfs.items():
        df.toPandas().to_sql(f"processed_{ticker}", conn, if_exists="replace", index=False)
    
    conn.close()
    print("All data successfully stored in SQLite database")

In [27]:
# Store raw and processed data in SQLite
print("Storing data in SQLite...")
store_data_to_sql(tweets_df, stock_dfs, processed_dfs)

Storing data in SQLite...
All data successfully stored in SQLite database


<h3 style="background-color:#2D3436;color:white;border-radius:8px;padding:15px">Working with Pandas DataFrame</h3>

In [28]:
# Convert processed Spark DataFrames to pandas DataFrames
def convert_to_pandas(processed_dfs):
    pandas_dfs = {}
    for ticker, df in processed_dfs.items():
        pandas_dfs[ticker] = df.toPandas()
    return pandas_dfs

In [29]:
pandas_result_dfs = convert_to_pandas(processed_dfs)

In [30]:
for ticker, df in pandas_result_dfs.items():
    print(ticker, 'Pandas Dataframe')
    display(df.head())
    print('\n')

AAPL Pandas Dataframe


,date,ticker,Open,High,Low,Close,Adj Close,Volume,tweet_count
0,2019-12-31,AAPL,72.482498,73.419998,72.379997,73.412498,71.520821,100805600,0
1,2020-01-02,AAPL,74.059998,75.150002,73.797501,75.087502,73.152649,135480400,3
2,2020-01-03,AAPL,74.287498,75.144997,74.125000,74.357498,72.441460,146322800,1
3,2020-01-06,AAPL,73.447502,74.989998,73.187500,74.949997,73.018677,118387200,2
4,2020-01-07,AAPL,74.959999,75.224998,74.370003,74.597504,72.675278,108872000,0




AMZN Pandas Dataframe


,date,ticker,Open,High,Low,Close,Adj Close,Volume,tweet_count
0,2019-12-31,AMZN,92.099998,92.663002,91.611504,92.391998,92.391998,50130000,0
1,2020-01-02,AMZN,93.750000,94.900497,93.207497,94.900497,94.900497,80580000,1
2,2020-01-03,AMZN,93.224998,94.309998,93.224998,93.748497,93.748497,75288000,0
3,2020-01-06,AMZN,93.000000,95.184502,93.000000,95.143997,95.143997,81236000,0
4,2020-01-07,AMZN,95.224998,95.694504,94.601997,95.343002,95.343002,80898000,4




MSFT Pandas Dataframe


...

,date,ticker,Open,High,Low,Close,Adj Close,Volume,tweet_count
0,2019-12-31,TSLA,27.000000,28.086000,26.805332,27.888666,27.888666,154285500,0
1,2020-01-02,TSLA,28.299999,28.713333,28.114000,28.684000,28.684000,142981500,1
2,2020-01-03,TSLA,29.366667,30.266666,29.128000,29.534000,29.534000,266677500,0
3,2020-01-06,TSLA,29.364668,30.104000,29.333332,30.102667,30.102667,151995000,7
4,2020-01-07,TSLA,30.760000,31.441999,30.224001,31.270666,31.270666,268231500,5




GOOGL Pandas Dataframe


,date,ticker,Open,High,Low,Close,Adj Close,Volume,tweet_count
0,2019-12-31,GOOGL,66.789497,67.032997,66.606499,66.969498,66.969498,19514000,0
1,2020-01-02,GOOGL,67.420502,68.433998,67.324501,68.433998,68.433998,27278000,0
2,2020-01-03,GOOGL,67.400002,68.687500,67.365997,68.075996,68.075996,23408000,0
3,2020-01-06,GOOGL,67.581497,69.916000,67.550003,69.890503,69.890503,46768000,0
4,2020-01-07,GOOGL,70.023003,70.175003,69.578003,69.755501,69.755501,34330000,0




FB Pandas Dataframe


,date,ticker,Open,High,Low,Close,Adj Close,Volume,tweet_count
0,2019-12-31,FB,204.000000,205.559998,203.600006,205.250000,205.250000,8953500,0
1,2020-01-02,FB,206.750000,209.789993,206.270004,209.779999,209.779999,12077100,0
2,2020-01-03,FB,207.210007,210.399994,206.949997,208.669998,208.669998,11188400,0
3,2020-01-06,FB,206.699997,212.779999,206.520004,212.600006,212.600006,17058900,0
4,2020-01-07,FB,212.820007,214.580002,211.750000,213.059998,213.059998,14912400,2
